In [1]:

# Packages that allow us to get information about objects:
import asdf
import copy
import os
import shutil
import glob
import warnings

# Numpy library:
import numpy as np
# Astropy tools:
from astropy.io import fits


In [2]:
import jwst

print('Pipeline version:', jwst.__version__)

# List of possible data quality flags
from jwst import datamodels
from jwst.datamodels import dqflags
# To read association file
import json

Pipeline version: 1.12.5


In [3]:
#Uncomment below two lines if you have not defined CRDS elsewhere
os.environ['CRDS_PATH']= '/data/public/jwst_crds'
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

## Stage 2

In [4]:
# The entire calwebb_image2 pipeline
from jwst.pipeline import calwebb_image2

# Individual steps that make up calwebb_image2
from jwst.background import BackgroundStep
from jwst.assign_wcs import AssignWcsStep
from jwst.flatfield import FlatFieldStep
from jwst.photom import PhotomStep
from jwst.resample import ResampleStep

In [5]:
input_dir='./stage1/'
output_dir = './stage2/'
output_dir_bkg = output_dir+'bkg/'
output_dir_assign_wcs = output_dir+'assign_wcs/'
output_dir_flatfield = output_dir+'flatfield/'
output_dir_photom = output_dir+'photom/'
output_dir_resample = output_dir+'resample/'

In [6]:
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir_bkg, exist_ok=True)
os.makedirs(output_dir_assign_wcs, exist_ok=True)
os.makedirs(output_dir_flatfield, exist_ok=True)
os.makedirs(output_dir_photom, exist_ok=True)
os.makedirs(output_dir_resample, exist_ok=True)

os.system('cp '+input_dir+'*_rate.fits '+output_dir+'')

0

In [7]:
# Create and open the association file and load into json object
os.system('asn_from_list -o level2_asn.json -r DMSLevel2bBase '+output_dir+'*_rate.fits')
asn_file ='level2_asn.json'

2024-01-03 21:13:35,526 - stpipe - WARNING - /data/home/mingyu/opt/miniconda3/envs/jwst_latest/lib/python3.10/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)



In [9]:
#If some parameters are known to have better results with certain value use the dictionary
parameter_dict = {"bkg_subtract": {"sigma":4,"output_dir": output_dir_bkg,"save_results": True},
                  "assign_wcs": {"output_dir": output_dir_assign_wcs,"save_results": True},
                  "flat_field": {"output_dir": output_dir_flatfield,"save_results": True},
                  "photom": {"output_dir": output_dir_photom,"save_results": True},
                  "resample": {"pixfrac": 1.0, "output_dir": output_dir_resample,"save_results": True},
                 }

In [74]:
call_output = calwebb_image2.Image2Pipeline.call(asn_file, output_dir=output_dir, save_results=True, steps=parameter_dict,logcfg='stage2-log.cfg')


2024-01-03 17:30:35,675 - stpipe - WARNING - /data/home/mingyu/opt/miniconda3/envs/jwst_latest/lib/python3.10/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)

2024-01-03 17:30:35,740 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.


2024-01-03 17:30:35,741 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2024-01-03 17:30:35,743 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2024-01-03 17:30:35,744 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2024-01-03 17:30:35,744 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2024-01-03 17:30:35,746 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2024-01-03 17:30:35,914 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('level2_asn.json',).
2024-01-03 17:30:35,918 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': './stage2/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_bsub': False, 'steps': {'bkg_subt

In [10]:
os.system('rm '+output_dir+'*_rate.fits')

0

## Stage 2.5

Destriping and background subtraction

In [12]:
from astroscrappy import detect_cosmics
import sep
from astropy.convolution import Tophat2DKernel
from scipy.ndimage import binary_dilation
import numpy as np
from astropy.table import Table
import pyregion
from astropy.stats import sigma_clipped_stats

def get_glob_bkg(dat, err):
    mask = (np.isfinite(err)==False)
    mask[:,:300]=True
    mask[:,1026:]=True
    ok = (mask==False)
    med_global = np.nanmedian(dat.astype('float')[ok])
    return med_global

def rm_strip(cal_file, rm_hz=False, rm_vt=False):
    with datamodels.open(cal_file) as dm:
        # Read science image and build weight
        sci_img = dm.data.copy()
        err_map = dm.err.copy()
        #wht = build_driz_weight(dm, weight_type='ivm', good_bits='~DO_NOT_USE+NON_SCIENCE')
        #mask = (wht==0)
        # Mask bad pixels
        mask = (np.isfinite(err_map)==False) | (dm.dq>4)
        err_map[mask] = np.inf
        # Get the global bkg values
        glob_bkg = get_glob_bkg(sci_img, err_map)
        # Subtract horizontal stripping
        if rm_hz:
            hz_strips = sigma_clipped_stats(sci_img, mask=mask, sigma=2, maxiters=None, cenfunc='median', stdfunc='std', axis=1)[1]
            sci_img = (sci_img.T-hz_strips).T
        # Subract vertical stripping
        if rm_vt:
            vt_strips = sigma_clipped_stats(sci_img, mask=mask, sigma=2, maxiters=None, cenfunc='median', stdfunc='std', axis=0)[1]
            sci_img -= vt_strips
        # If not horizontal nor vertical, subtract a constant global background
        if (rm_hz or rm_vt)==False:
            sci_img -= glob_bkg
        else:
            # Set all nan (may be caused by strip removal)
            mask[np.isnan(sci_img)] = True
            sci_img[np.isnan(sci_img)] = 0
            err_map[mask] = np.inf
        # Replace the old data
        dm.data = sci_img
        #dm.err = err_map
        dm.save(cal_file.replace('.fits','')+'_stprm.fits', overwrite=True)
    return glob_bkg

def det_src(img, sigma_threshold=2, bw=72, bh=72, fw=5, fh=5,
            clean_edge=True, cr_sigclip=1., minarea=5, r_dilate=-1,
            kernel=np.array([[1,2,1], [2,4,2], [1,2,1]])):
    # mask_threshold=0.3, no longer used
    # Open data model
    #with datamodels.open(img) as dm:
        # Read science image and build weight
        #dat = dm.data.copy()
        #wht = build_driz_weight(dm, weight_type='ivm', good_bits='~DO_NOT_USE+NON_SCIENCE')
        # subtract a constant background first 
        # This is no longer needed becuase we remove the stripes before
        #dat -= get_glob_bkg(dat, dm.err)
    
    # Check input type
    if type(img) == str:
        # Read data
        dat = fits.getdata(img, 'sci')
        err = fits.getdata(img, 'err')
        dq = fits.getdata(img, 'dq')
    elif type(img) == jwst.datamodels.ImageModel:
        dat = img.data.copy()
        err = img.err.copy()
        dq = img.dq.copy()
    # Mask cosmic rays
    cr_map = detect_cosmics(dat, sigclip=cr_sigclip)[0]*1
    err[np.where(cr_map)] = np.inf
    err[dq>4] = np.inf
    # subtract a background, then find objects.     
    #mask = (wht < np.max(wht)* mask_threshold) 
    wht = 1./err**2
    mask = (np.isfinite(err)==False)
    wht[mask] = np.nan
    wht /= np.nanmedian(wht)
    # Estimate a spatial dependent background
    bkg = sep.Background(dat.astype('float'), mask=mask, bw=bw, bh=bh, fw=fw, fh=fh)
    bkg.back()[mask] = np.nan
    # identify objects
    wdat = (dat-bkg.back())  * wht
    wdat[mask] = np.nan
    wbkg = sep.Background(wdat, mask=mask, bw=bw, bh=bh, fw=fw, fh=fh)  
    objects = sep.extract(wdat, sigma_threshold, mask=mask, filter_kernel=kernel,
                          err=wbkg.globalrms, segmentation_map=True, minarea=minarea)
    src, seg = objects[0], objects[1]
    # Convert src to astropy table format
    src = Table(src)
    # Add an ID column 
    src.add_column( np.arange(len(src))+1, name='id' )
    # Clean sources close to detector edge if needed
    if clean_edge: 
        # Read the bad master region file
        regs = pyregion.open("../data/master_noisy_region.reg")
        # Iterate over each bad region
        bad_src_idxs = []
        for reg in regs:
            # Get the four points
            x1, x2 = reg.coord_list[0]-reg.coord_list[2]/2, reg.coord_list[0]+reg.coord_list[2]/2
            y1, y2 = reg.coord_list[1]-reg.coord_list[3]/2, reg.coord_list[1]+reg.coord_list[3]/2
            # Get sources in the region
            bad_src_idxs = np.append(bad_src_idxs,
                    np.where( (src['x']>x1) & (src['x']<x2) & (src['y']>y1) & (src['y']<y2) )[0] )
        bad_src_idxs = np.sort(np.unique(bad_src_idxs)).astype(int)
        # Remove the bad sources in both catalog and segmentaion
        good_src_idxs = np.delete(np.arange(len(src)), bad_src_idxs)
        src = src[good_src_idxs]
        for bad_id in bad_src_idxs+1:
            seg[seg==bad_id] = 0
    # Dilate the segmentaion? (but all pixel will be set to 1)
    if r_dilate>0:
        footprint = Tophat2DKernel(radius=r_dilate)
        seg = binary_dilation(seg, footprint.array).astype(int)
            
    return src, dat, wht, bkg, seg

def super_bg(files, sigma_threshold=2, bw=72, bh=72, fw=5, fh=5, minarea=20,
             kernel=np.array([[1,2,1], [2,4,2], [1,2,1]]), r_dilate=10,
             clean_edge=True, write_bg=False):
    # read in each image.  Identify objects and mask with background from SEP
    x, y = np.shape( fits.getdata(files[0]) )
    darr = np.zeros( [len(files), x, y] )
    # Check number of files 
    if len(files)<2:
        raise Exception("Too few input files?!")
    elif len(files)==2:
        print('Warning: building master background from a single file')
        # Use different parameters if building from a single image
        clean_edge = False
        r_dilate = -1
        minarea = 5
        sigma_threshold = 1.2
    else:
        print('Building master background from a %d files' %(len(files)-1))
    # Iterate over each file to get the background     
    for i,f in enumerate(files):
        # Detect sources 
        src, dat, wht, bkg, seg = det_src(f, sigma_threshold=sigma_threshold,
            r_dilate=r_dilate, bw=bw, bh=bh, fw=fw, fh=fh, cr_sigclip=1.,
            minarea=minarea, kernel=kernel, clean_edge=clean_edge)
        print("%d src detected!" %len(src))
        # Fill in NaN or bkg, depending on number of files available
        if len(files)>2:
            dat[seg > 0] = np.nan
        else:
            dat[seg > 0] = bkg.back()[seg > 0]        
        # Save the data
        darr[i] = dat
    # Subtract background for each file
    for idx0, f0 in enumerate(files):
        #take median 
        oth_idxs = np.delete(range(len(files)), idx0)
        dmed = np.nanmedian(darr[oth_idxs], axis=0)
        # Set nan to 0 to avoid NaN in the result
        # dmed[np.isnan( dmed )] = 0
        # subtract and writeout
        with fits.open(f0) as hdu:
            # Subtract the background
            hdu['sci'].data -= dmed
            mask = (np.isfinite(hdu['err'].data)==False) | np.isnan(hdu['sci'].data) | hdu['dq'].data>4
            hdu['sci'].data[mask] = 0
            hdu['err'].data[mask] = np.inf
            hdu.writeto( f0.replace('.fits','')+'_bgsub.fits', overwrite=True)
            # Write background if needed
            if write_bg: 
                hdu['sci'].data = dmed
                hdu.writeto( f0.replace('.fits','')+'_bg.fits', overwrite=True)
        print(f0.replace('.fits','')+'_bgsub.fits' + "  Done!")
    
    return 

In [13]:
def group_imgs(all_files, prop):
    # Adapted from CEERS MIRI notebook https://github.com/ceers/ceers-miri
    # Extract the needed property for each file
    prop_strs = []
    for f in all_files:
        hd = fits.getheader(f)
        # print proposal id
        print(f, hd['PROGRAM'], hd['TARGPROP'], hd['FILTER'], hd['READPATT'], hd['EFFEXPTM'])
        if prop=="band":
            prop_strs.append( hd['FILTER'] )
        elif prop=="field":
            prop_strs.append( hd['TARGPROP'] )
        elif prop=="field_band":
            prop_strs.append( hd['TARGPROP'] + '_' + hd['FILTER'] )
        elif prop=='propid_band_exp':
            prop_strs.append( str(hd['PROGRAM']) + '_' + hd['FILTER'] + '_' + f"{hd['EFFEXPTM']:.0f}s" )
        else:
            ValueError("prop unknown")

    # Group the files 
    groups = {}
    unq_prop_strs = list(set(prop_strs))
    for prop_str in unq_prop_strs:
        group = []
        # Find the files with the prop_str
        for file_idx, buf in enumerate(prop_strs):
            if buf==prop_str: group.append( all_files[file_idx] )
        groups[prop_str] = group
        
    return groups

In [14]:
output_dir = './stage2.5/'
os.makedirs(output_dir, exist_ok=True)

input_dir='./stage2/'
os.system('cp '+input_dir+'*_cal.fits '+output_dir+'')
list_files=glob.glob(output_dir+'*_cal.fits')

In [15]:
for file in list_files:
    # Run the strip removal 
    rm_strip(file, rm_hz=True, rm_vt=True)


2024-01-03 21:14:13,661 - stpipe - WARNING - /data/home/mingyu/opt/miniconda3/envs/jwst_latest/lib/python3.10/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(data_clipped, axis=axis)

2024-01-03 21:14:13,662 - stpipe - WARNING - /data/home/mingyu/opt/miniconda3/envs/jwst_latest/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,

2024-01-03 21:14:13,714 - stpipe - WARNING - /data/home/mingyu/opt/miniconda3/envs/jwst_latest/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,

2024-01-03 21:14:14,206 - stpipe - WARNING - /data/home/mingyu/opt/miniconda3/envs/jwst_latest/lib/python3.10/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
  mean = np.nanmea

In [29]:
# Group the files by target name and filter band
list_files_stprm = glob.glob(output_dir+'*_cal_stprm.fits')
groups = group_imgs(list_files_stprm, 'propid_band_exp')
groups

./stage2.5/jw01837014001_02101_00001_mirimage_cal_stprm.fits 01837 UDS-2-TILE-6 F770W FASTR1 444.006
./stage2.5/jw01837014001_02101_00002_mirimage_cal_stprm.fits 01837 UDS-2-TILE-6 F770W FASTR1 444.006
./stage2.5/jw01837014001_04101_00001_mirimage_cal_stprm.fits 01837 UDS-2-TILE-6 F770W FASTR1 444.006
./stage2.5/jw01837014001_04101_00002_mirimage_cal_stprm.fits 01837 UDS-2-TILE-6 F770W FASTR1 444.006
./stage2.5/jw01837014001_06101_00001_mirimage_cal_stprm.fits 01837 UDS-2-TILE-6 F1800W FASTR1 446.781
./stage2.5/jw01837014001_06101_00002_mirimage_cal_stprm.fits 01837 UDS-2-TILE-6 F1800W FASTR1 446.781
./stage2.5/jw01837014001_08101_00001_mirimage_cal_stprm.fits 01837 UDS-2-TILE-6 F1800W FASTR1 446.781
./stage2.5/jw01837014001_08101_00002_mirimage_cal_stprm.fits 01837 UDS-2-TILE-6 F1800W FASTR1 446.781
./stage2.5/jw01837022001_02101_00001_mirimage_cal_stprm.fits 01837 UDS-2-TILE-22 F770W FASTR1 444.006
./stage2.5/jw01837022001_02101_00002_mirimage_cal_stprm.fits 01837 UDS-2-TILE-22 F770W

{'01837_F770W_444s': ['./stage2.5/jw01837014001_02101_00001_mirimage_cal_stprm.fits',
  './stage2.5/jw01837014001_02101_00002_mirimage_cal_stprm.fits',
  './stage2.5/jw01837014001_04101_00001_mirimage_cal_stprm.fits',
  './stage2.5/jw01837014001_04101_00002_mirimage_cal_stprm.fits',
  './stage2.5/jw01837022001_02101_00001_mirimage_cal_stprm.fits',
  './stage2.5/jw01837022001_02101_00002_mirimage_cal_stprm.fits',
  './stage2.5/jw01837022001_04101_00001_mirimage_cal_stprm.fits',
  './stage2.5/jw01837022001_04101_00002_mirimage_cal_stprm.fits',
  './stage2.5/jw01837023001_02101_00001_mirimage_cal_stprm.fits',
  './stage2.5/jw01837023001_02101_00002_mirimage_cal_stprm.fits',
  './stage2.5/jw01837023001_04101_00001_mirimage_cal_stprm.fits',
  './stage2.5/jw01837023001_04101_00002_mirimage_cal_stprm.fits',
  './stage2.5/jw01837028001_02101_00001_mirimage_cal_stprm.fits',
  './stage2.5/jw01837028001_04101_00001_mirimage_cal_stprm.fits',
  './stage2.5/jw01837028001_04101_00002_mirimage_cal_stp

In [30]:
# Subtract super background
for group_name, group_files in groups.items():
    print(group_name, group_files)
    print(f"processing {len(group_files)} files for sky subtraction of {group_name}") 
    super_bg(group_files, write_bg=True, clean_edge=False)


01837_F770W_444s ['./stage2.5/jw01837014001_02101_00001_mirimage_cal_stprm.fits', './stage2.5/jw01837014001_02101_00002_mirimage_cal_stprm.fits', './stage2.5/jw01837014001_04101_00001_mirimage_cal_stprm.fits', './stage2.5/jw01837014001_04101_00002_mirimage_cal_stprm.fits', './stage2.5/jw01837022001_02101_00001_mirimage_cal_stprm.fits', './stage2.5/jw01837022001_02101_00002_mirimage_cal_stprm.fits', './stage2.5/jw01837022001_04101_00001_mirimage_cal_stprm.fits', './stage2.5/jw01837022001_04101_00002_mirimage_cal_stprm.fits', './stage2.5/jw01837023001_02101_00001_mirimage_cal_stprm.fits', './stage2.5/jw01837023001_02101_00002_mirimage_cal_stprm.fits', './stage2.5/jw01837023001_04101_00001_mirimage_cal_stprm.fits', './stage2.5/jw01837023001_04101_00002_mirimage_cal_stprm.fits', './stage2.5/jw01837028001_02101_00001_mirimage_cal_stprm.fits', './stage2.5/jw01837028001_04101_00001_mirimage_cal_stprm.fits', './stage2.5/jw01837028001_04101_00002_mirimage_cal_stprm.fits', './stage2.5/jw01837028

2024-01-03 20:30:37,454 - stpipe - WARNING - /tmp/ipykernel_794274/1658824691.py:161: RuntimeWarning: All-NaN slice encountered
  dmed = np.nanmedian(darr[oth_idxs], axis=0)



./stage2.5/jw01837014001_06101_00001_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837014001_06101_00002_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837014001_08101_00001_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837014001_08101_00002_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837022001_06101_00001_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837022001_06101_00002_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837022001_08101_00001_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837022001_08101_00002_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837023001_06101_00001_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837023001_06101_00002_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837023001_08101_00001_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837023001_08101_00002_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837028001_06101_00001_mirimage_cal_stprm_bgsub.fits  Done!
./stage2.5/jw01837028001_06101_00002_m

## Stage 3

In [16]:
# The entire calwebb_image3 pipeline
from jwst.pipeline import calwebb_image3

# Individual steps that make up calwebb_image3
from jwst.tweakreg import TweakRegStep
from jwst.skymatch import SkyMatchStep
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.source_catalog import SourceCatalogStep
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base


In [17]:
output_dir = './stage3/'
output_dir_tweakreg = output_dir+'tweakreg/'
output_dir_skymatch = output_dir+'skymatch/'
output_dir_outlier_detection = output_dir+'outlier_detection/'
output_dir_resample = output_dir+'resample/'
output_dir_source_catalog = output_dir+'source_catalog/'

os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir_tweakreg, exist_ok=True)
os.makedirs(output_dir_skymatch, exist_ok=True)
os.makedirs(output_dir_outlier_detection, exist_ok=True)
os.makedirs(output_dir_resample, exist_ok=True)
os.makedirs(output_dir_source_catalog, exist_ok=True)

In [18]:
input_dir='./stage2.5/'
os.system('cp '+input_dir+'*_cal_stprm_bgsub.fits '+output_dir+'')


0

In [19]:
file_list_cal_stprm_bgsub = glob.glob(output_dir+'*_cal_stprm_bgsub.fits')
level3_groups = group_imgs(file_list_cal_stprm_bgsub, 'field_band')
level3_groups

./stage3/jw01837014001_02101_00001_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-6 F770W FASTR1 444.006
./stage3/jw01837014001_02101_00002_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-6 F770W FASTR1 444.006
./stage3/jw01837014001_04101_00001_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-6 F770W FASTR1 444.006
./stage3/jw01837014001_04101_00002_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-6 F770W FASTR1 444.006
./stage3/jw01837014001_06101_00001_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-6 F1800W FASTR1 446.781
./stage3/jw01837014001_06101_00002_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-6 F1800W FASTR1 446.781
./stage3/jw01837014001_08101_00001_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-6 F1800W FASTR1 446.781
./stage3/jw01837014001_08101_00002_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-6 F1800W FASTR1 446.781
./stage3/jw01837022001_02101_00001_mirimage_cal_stprm_bgsub.fits 01837 UDS-2-TILE-22 F770W FASTR1 444.006
./stage3/jw01837022001_02101_00002_mirimage_cal_st

{'UDS-2-TILE-7_F770W': ['./stage3/jw01837023001_02101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837023001_02101_00002_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837023001_04101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837023001_04101_00002_mirimage_cal_stprm_bgsub.fits'],
 'UDS-2-TILE-6_F770W': ['./stage3/jw01837014001_02101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837014001_02101_00002_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837014001_04101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837014001_04101_00002_mirimage_cal_stprm_bgsub.fits'],
 'UDS-2-TILE-22_F770W': ['./stage3/jw01837022001_02101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837022001_02101_00002_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837022001_04101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837022001_04101_00002_mirimage_cal_stprm_bgsub.fits'],
 'UDS-2-TILE-6_F1800W': ['./stage3/jw01837014001_06101_00001_mirimage_cal_stprm_bgsub.fits

In [26]:
def merge_group(groups, group_name_list, new_group_name):
    # Merge a list of groups into a new group
    # Get the files
    files = []
    for group_name in group_name_list:
        files.extend( level3_groups[group_name] )
    # Add to the level3_groups
    level3_groups[new_group_name] = files
    return 

merge_group(level3_groups, ['CWEBTILE-5-7_F770W', 'COSMOS-2-TILE-7_F770W', 'CWEBTILE-5-6_F770W'], 'CosmicOwl_F770W')
merge_group(level3_groups, ['COSMOS-2-TILE-7_F1800W'], 'CosmicOwl_F1800W')

level3_groups

{'UDS-2-TILE-7_F770W': ['./stage3/jw01837023001_02101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837023001_02101_00002_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837023001_04101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837023001_04101_00002_mirimage_cal_stprm_bgsub.fits'],
 'UDS-2-TILE-6_F770W': ['./stage3/jw01837014001_02101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837014001_02101_00002_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837014001_04101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837014001_04101_00002_mirimage_cal_stprm_bgsub.fits'],
 'UDS-2-TILE-22_F770W': ['./stage3/jw01837022001_02101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837022001_02101_00002_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837022001_04101_00001_mirimage_cal_stprm_bgsub.fits',
  './stage3/jw01837022001_04101_00002_mirimage_cal_stprm_bgsub.fits'],
 'UDS-2-TILE-6_F1800W': ['./stage3/jw01837014001_06101_00001_mirimage_cal_stprm_bgsub.fits

In [21]:
# Create and open the association file and load into json object
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3 import Asn_Lv3Image

group = 'CosmicOwl_F770W'
asn = asn_from_list.asn_from_list(level3_groups[group], rule=Asn_Lv3Image, 
                product_name=group, asn_type="image3")
asn_file_path = f'{group}_level3_asn.json'
with open(asn_file_path, 'w') as outfile:
    name, serialized = asn.dump(format='json')
    outfile.write(serialized)


#If some parameters are known to have better results with certain value use the dictionary
parameter_dict = {"tweakreg": {"snr_threshold": 3.0, "brightest": 1000, "separation":0.1, "searchrad":1, "tolerance":0.5, "abs_minobj":4, "expand_refcat":True, "output_dir": output_dir_tweakreg,"save_results": True,"abs_refcat":'GAIADR3'}, #kernel_fwhm can also be added for better centroiding
                  "skymatch": {"skip":False,"skymethod":'local',"output_dir": output_dir_skymatch,"save_results": True,"subtract":True,"match_down":True},
                  "outlier_detection": {'snr': '3.0 2.0', 'scale': '0.4 0.3', "output_dir": output_dir_outlier_detection,"save_results": True},
                  "resample": {"pixfrac": 0.8,"pixel_scale":0.1,"rotation":0,"output_dir": output_dir_resample,"save_results": True}, #pixfrac can be higher if one wants lesser resolution
                  "source_catalog": {"snr_threshold": 5.0,"output_dir": output_dir_source_catalog,"save_results": True}, 
                 }

call_output = calwebb_image3.Image3Pipeline.call(asn_file_path, output_dir=output_dir, save_results=True, steps=parameter_dict,logcfg='stage3-log.cfg')


2024-01-03 21:14:42,332 - stpipe - WARNING - /data/home/mingyu/opt/miniconda3/envs/jwst_latest/lib/python3.10/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)



In [27]:
group = 'CosmicOwl_F1800W'
asn = asn_from_list.asn_from_list(level3_groups[group], rule=Asn_Lv3Image, 
                product_name=group, asn_type="image3")
asn_file_path = f'{group}_level3_asn.json'
with open(asn_file_path, 'w') as outfile:
    name, serialized = asn.dump(format='json')
    outfile.write(serialized)


#If some parameters are known to have better results with certain value use the dictionary
parameter_dict = {"tweakreg": {"snr_threshold": 3.0, "brightest": 1000, "separation":0.1, "searchrad":1, "tolerance":0.5, "abs_minobj":4, "expand_refcat":True, "output_dir": output_dir_tweakreg,"save_results": True,"abs_refcat":'GAIADR3'}, #kernel_fwhm can also be added for better centroiding
                  "skymatch": {"skip":False,"skymethod":'local',"output_dir": output_dir_skymatch,"save_results": True,"subtract":True,"match_down":True},
                  "outlier_detection": {'snr': '3.0 2.0', 'scale': '0.4 0.3', "output_dir": output_dir_outlier_detection,"save_results": True},
                  "resample": {"pixfrac": 0.8,"pixel_scale":0.1,"rotation":0,"output_dir": output_dir_resample,"save_results": True}, #pixfrac can be higher if one wants lesser resolution
                  "source_catalog": {"snr_threshold": 5.0,"output_dir": output_dir_source_catalog,"save_results": True}, 
                 }

call_output = calwebb_image3.Image3Pipeline.call(asn_file_path, output_dir=output_dir, save_results=True, steps=parameter_dict,logcfg='stage3-log.cfg')


2024-01-03 22:34:38,775 - stpipe - WARNING - /data/home/mingyu/opt/miniconda3/envs/jwst_latest/lib/python3.10/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)

2024-01-03 22:34:38,925 - stpipe - INFO - PARS-TWEAKREGSTEP parameters found: /data/public/jwst_crds/references/jwst/miri/jwst_miri_pars-tweakregstep_0015.asdf
2024-01-03 22:34:38,938 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: /data/public/jwst_crds/references/jwst/miri/jwst_miri_pars-outlierdetectionstep_0036.asdf
2024-01-03 22:34:38,949 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /data/public/jwst_crds/references/jwst/miri/jwst_miri_pars-sourcecatalogstep_0016.asdf
2024-01-03 22:34:38,966 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2024-01-03 22:34:38,967 - stpipe.Image3Pipeline.assign_mtwcs -